Libraries

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

Database Connection

In [ ]:
db = mysql.connector.connect(
    host="localhost",
    user="user",
    password="password",
    port="3306",
    database="realestatepredictor"
)

Get Data

In [ ]:
# Create a cursor object
cursor = db.cursor()

# Execute the query
cursor.execute("SELECT * FROM homegate")

# Fetch all the records
data = cursor.fetchall()

# Convert the data into a DataFrame
df = pd.DataFrame(data, columns=['id', 'homegateid', 'price', 'rooms', 'floor', 'livingSpace', 'street', 'latitude', 'longitude', 'locality', 'postalcode'])

# Close the cursor and connection
cursor.close()
db.close()

Null and empty values

In [ ]:
# Handle null / empty values
df = df.dropna()

Descriptive Statistics

In [ ]:
print(df.describe())

Visualize Data

In [ ]:
# Show the distribution of the different home prices
sns.displot(df['price'], kde=True)
plt.show()

# Show scatter plots to visualize the data
sns.pairplot(df[['price', 'rooms', 'floor', 'livingSpace', 'street', 'latitude', 'longitude', 'locality', 'postalcode']])
plt.show()

# Show a correlation heatmap
sns.heatmap(df[['price', 'rooms', 'floor', 'livingSpace', 'latitude', 'longitude']].corr(), annot=True, cmap='coolwarm')
plt.show()

# Visualize the boxplot for the price
sns.boxplot(x=df['price'])
plt.show()

# Visualize boxplots for all numeric columns
"""for column in df.select_dtypes(include=[np.number]).columns:
    plt.figure(figsize=(5, 5))
    sns.boxplot(x=df[column])
    plt.title(f"Boxplot of {column}")
    plt.show()"""

GIS

In [ ]:
import folium

# Create a map centered around the mean latitude and longitude values
m = folium.Map(location=[df['latitude'].mean(), df['longitude'].mean()], zoom_start=8)

# Add a marker for each home to the map
for index, row in df.iterrows():
    folium.Marker([row['latitude'], row['longitude']]).add_to(m)

# Display the map
m